In [1]:
# importing Libaries

import time
import statistics
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from scipy.stats import mode

In [2]:
# Load the image in black and white (0 - b/w, 1 - color).
def inner_box(img,count,height,number):
    # img is image
    # Count is section 0-5
    # height is image height
    # number is image number from the batch

    #Invert the image to be white on black for compatibility with findContours function.
    imgray = 255 - img

# Using different combination of dilation and blur to get as much as contours possible

    # rects1
    
    kernel = np.ones((1,10),np.uint8)
    imgray1 = cv2.dilate(imgray,kernel,iterations = 1)
    kernel = np.ones((10,1),np.uint8)
    imgray1 = cv2.dilate(imgray1,kernel,iterations = 1)
    thresh1 = cv2.adaptiveThreshold(imgray1, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)

    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects1 = [cv2.boundingRect(cnt) for cnt in contours]


    # rects2 imp
    imgray2 = cv2.blur(imgray,(10,3))
    thresh2 = cv2.adaptiveThreshold(imgray2, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)

    
    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects2 = [cv2.boundingRect(cnt) for cnt in contours]

    # rects3 imp
    imgray3 = cv2.blur(imgray,(3,10))
    thresh3 = cv2.adaptiveThreshold(imgray3, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
    
    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh3, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects3 = [cv2.boundingRect(cnt) for cnt in contours]

    #rects4 imp
    kernel = np.ones((2,7),np.uint8)
    imgray4 = cv2.dilate(imgray,kernel,iterations = 1)
    thresh4 = cv2.adaptiveThreshold(imgray4, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
    
    #Find all the contours in thresh.
    contours, hierarchy = cv2.findContours(thresh4, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects4 = [cv2.boundingRect(cnt) for cnt in contours]

    
    #rects5 imp
    kernel = np.ones((7,2),np.uint8)
    imgray5 = cv2.dilate(imgray,kernel,iterations = 1)
    thresh5 = cv2.adaptiveThreshold(imgray5, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,2)
    
    #Find all the contours in thresh.
    contours, hierarchy = cv2.findContours(thresh5, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    rects5 = [cv2.boundingRect(cnt) for cnt in contours]
    
    
# Defining function to filter sections for different size sections

    def filterrects1(raw_rects,rects):
        for i in raw_rects:
            if (abs(i[2]-i[3])<10):
                if (i[2]>height/75 and i[2]<height/40 and i[3]>height/75 and i[3]<height/40 and i not in rects):
                    rects.append(i)
        return(rects)
    
    def filterrects2(raw_rects,rects):
        for i in raw_rects:
            if (i[2]>height/40 and i[2]<height/25 and i[3]>height/40 and i[3]<height/25 and i not in rects):
                rects.append(i)
        return(rects)
    
# checking the section number and calling function accordingly.
    if count!=0:
        rects=[]
        rects=filterrects1(rects1,rects)
        rects=filterrects1(rects2,rects)
        rects=filterrects1(rects3,rects)
        rects=filterrects1(rects4,rects)
        rects=filterrects1(rects5,rects)
                    
    else:
        rects=[]
        rects=filterrects2(rects1,rects)
        rects=filterrects2(rects2,rects)
        rects=filterrects2(rects3,rects)
        rects=filterrects2(rects4,rects)
        rects=filterrects2(rects5,rects)

    rects_final=rects.copy()
    
# Removing the sections which are overlapping
    for i in range(0,len(rects)-1):
        try:
            for j in range(i+1,len(rects)):
                if (abs(rects[i][0] - rects[j][0]) < 15 and abs(rects[i][1] - rects[j][1]) < 15):
                    if (rects[i][2] < rects[j][2] and rects[i][3] < rects[j][3]):
                        rects_final.remove(rects[j])
                    else:
                        rects_final.remove(rects[i])
        except:
            continue
#     print(len(rects_final))
    rects=rects_final.copy()

    #Calculate the combined bounding rectangle points.
    top_x = min([x for (x, y, w, h) in rects])
    top_y = min([y for (x, y, w, h) in rects])
    bottom_x = max([x+w for (x, y, w, h) in rects])
    bottom_y = max([y+h for (x, y, w, h) in rects])


    from scipy.stats import mode
    h_mode = mode([h for (x, y, w, h) in rects])[0]
    w_mode = mode([w for (x, y, w, h) in rects])[0]
    n=int(((bottom_y-top_y)/h_mode))

# Sorting the sections in increasing order as they occur in image
    import operator
    temp=[]
    b = [[] for i in range(0, n+3)]
    for j in range (1,n+3):
        for i in rects:
            x=i[0]
            y=i[1]
            w=i[2]
            h=i[3]
            
        # Keeping constraint on y and sorting as per x pixels
            if ((y > bottom_y-h_mode*j-h_mode/2.5) and (i not in temp)):
                b[j].append(i)
                temp.append(i)
            b[j]=sorted(b[j],key=lambda x:(-x[0]))
    b.pop(0)
    tar_dir=r"C:\Users\Rahul\Desktop\Capstone\form\Inner\\"+str(number)+"\\"+str(count)+"\\"
    if not os.path.exists(tar_dir):
        #print("here2")
        os.makedirs(tar_dir) 

# Traversing through each contour and saving each contour
    count1=0
    for i in b:
        len1=len(i)
        for j in i:
            try:
                x=j[0]
                y=j[1]
                w=j[2]
                h=j[3]
                cv2.imwrite(tar_dir+str(count1)+".jpg",img[y-2:y+h+2,x-2:x+w+2])
                time.sleep(0.01)
                count1+=1
            except:
                continue

In [3]:
# to check if all major boxes came and try to get correct one by hard code if some error.

def validrects(rects3):
    if len (rects3)!=6:
        rects_f=[]
        
        # saving all cooredinates of large box by manuualy calculating x,y,w,h using height of whole image and 1st box
        first=rects3[0]
        second=(first[0],first[1]+first[3]+round(height/159.41),round(first[2]*2.05),round(height/11.13))
        third=(first[0],second[1]+second[3]+round(height/159.41),round(first[2]*2.05),round(height/4.45))
        fourth=(first[0]+(round(first[2]*2.05*5/35)),third[1]+third[3]+round(height/159.41),round(first[2]*2.05),round(height/8.37))
        fifth=(first[0]+(round(first[2]*2.05*5/35)),fourth[1]+fourth[3]+round(height/159.41),round(first[2]*2.05),round(height/11.89))
        sixth=(first[0]+(round(first[2]*2.05*4/35)),fifth[1]+fifth[3]+round(height/159.41),round(first[2]*2.05),round(height/8.77))
        rects_f=[first,second,third,fourth,fifth,sixth]
        return rects_f
    
    elif (len (rects3)==6 and  rects3[1][3]/rects3[0][3] > 1.25 and rects3[2][3]/rects3[0][3] > 3.25 and 
    rects3[3][3]/rects3[0][3] > 1.50 and rects3[4][3]/rects3[0][3] > 1.25 and rects3[5][3]/rects3[0][3] > 1.50):
        return rects3
    else:
        rects_f=[]
        # saving all cooredinates of large box by manuualy calculating x,y,w,h using height of whole image and 1st box
        first=rects3[0]
        second=(first[0],first[1]+first[3]+round(height/159.41),round(first[2]*2.05),round(height/11.13))
        third=(first[0],second[1]+second[3]+round(height/159.41),round(first[2]*2.05),round(height/4.45))
        fourth=(first[0]+(round(first[2]*2.05*5/35)),third[1]+third[3]+round(height/159.41),round(first[2]*2.05),round(height/8.37))
        fifth=(first[0]+(round(first[2]*2.05*5/35)),fourth[1]+fourth[3]+round(height/159.41),round(first[2]*2.05),round(height/11.89))
        sixth=(first[0]+(round(first[2]*2.05*4/35)),fifth[1]+fifth[3]+round(height/159.41),round(first[2]*2.05),round(height/8.77))
        rects_f=[first,second,third,fourth,fifth,sixth]
        return rects_f
        return rects3
        

In [4]:
#Load the image in black and white (0 - b/w, 1 - color).
for number in range(1,13):
    img = cv2.imread(r'C:\Users\Rahul\Desktop\Capstone\fwddocuments\\'+str(number)+'.jpg', 0)

    #Get the height and width of the image.
    height, width = img.shape[:2]

    #Invert the image to be white on black for compatibility with findContours function.
    imgray = 255 - img

    # Remove noise using dilate and thershold
    thresh = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,2)
    kernel = np.ones((2,2),np.uint8)
    thresh = cv2.dilate(thresh,kernel,iterations = 1)

    #Find all the contours in thresh. 
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    #Find bounding rectangles for each contour.
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    rects2=[]
    count=0
    for i in rects:
        # hierarchy [Next, Previous, First_Child, Parent]
        if (hierarchy[0][count][2]>0 and hierarchy[0][count][3]<0):   # condn that contour is parent and have child.
            if (i[2]>thresh.shape[1]/3 and i[2] < thresh.shape[1]):   # condn on width of contours
                rects2.append(i)
        count+=1
    h_mode = mode([h for (x, y, w, h) in rects2])[0]

    # to remove noisy contour i.e. which are small in height if these
    if h_mode < height/20:
        rects2=[]
        count=0
        for i in rects:
            if (hierarchy[0][count][2]>0):
                if (i[2]>thresh.shape[1]/3 and i[2] < thresh.shape[1] and i[3] > h_mode*1.2 ):
                    rects2.append(i)
            count+=1

    # to remove duplicate contours having very less coordinate difference
    rects3=rects2.copy()
    for i in range(0,len(rects2)-1):
        try:
            for j in range(i+1,len(rects2)):
                if (rects2[i][1] - rects2[j][1] <50):
                    if rects2[i][2]>rects2[j][2]:
                        value=rects2[j]
                        rects3.remove(value)
                    else :
                        value=rects2[i]
                        rects3.remove(value)
        except:
            continue

    # sort in correct order
    rects3=sorted(rects3,key=lambda x:(x[1]))
    rects3=validrects(rects3)
    # saving all
    count=0

    for j in rects3:
        try:
            x=j[0]
            y=j[1]
            w=j[2]
            h=j[3]
            inner_box(img[y:y+h,x:x+w],count,height,number)
            if not os.path.exists(r"C:\Users\Rahul\Desktop\Capstone\form\Outer\\"+str(number)):
                os.makedirs(r"C:\Users\Rahul\Desktop\Capstone\form\Outer\\"+str(number)) 
            cv2.imwrite(r"C:\Users\Rahul\Desktop\Capstone\form\Outer\\"+str(number)+"\\img"+str(count)+".jpg",img[y:y+h,x:x+w])
            time.sleep(0.001)
            count+=1
        except:
            continue

6
87
239
116
29
56
6
87
239
114
30
55
6
88
232
115
30
54
6
87
237
115
30
56
6
84
229
109
32
54
6
86
238
116
31
55
6
87
237
116
30
57
6
87
237
116
30
57
6
87
239
116
30
55
6
87
238
116
30
55
6
87
239
116
30
56
6
87
238
116
30
57
